# Implementation de la class abstraite Problem


´´´
    grille acutuelle : 
    4 3 2 9
    4 3 2 0
    5 1 1 6
    5 1 1 0
    8 7 A A
    
    initial = [
    ['4', '3', '2', '9'],
    ['4', '3', '2', '0'],
    ['5', '1', '1', '6'],
    ['5', '1', '1', '0'],
    ['8', '7', 'A', 'A']
    ]
    
    goal = [
    [., ., ., .],
    [., ., ., .],
    [., ., ., .],
    [., 1, 1, .],
    [., 1, 1, .],
    ]
´´´

# J'ai tois types de pièces : 

---

## **1. Pièces Verticales (Type 1)**

### **Identifier la pièce :**
- Une pièce verticale est détectée si :  
  - `i+1 < row` et `n[i,j] == n[i+1,j]`  
  - `i-1 >= 0` et `n[i,j] == n[i-1,j]`  

### **Actions possibles :**
- **Aller en haut** : si `i-1 >= 0` et `n[i-1,j] == 0`
- **Aller en bas** : si `i+2 < row` et `n[i+2,j] == 0`
- **Aller à droite** : si `j+1 < col` et `i+1 < row` et `n[i,j+1] == 0` et `n[i+1,j+1] == 0`
- **Aller à gauche** : si `j-1 >= 0` et `i+1 < row` et `n[i,j-1] == 0` et `n[i+1,j-1] == 0`

---

## **2. Pièces Horizontales (Type 2)**

### **Identifier la pièce :**
- Une pièce horizontale est détectée si :  
  - `j+1 < col` et `n[i,j] == n[i,j+1]`
  - `j-1 >= 0` et `n[i,j] == n[i,j-1]`

### **Actions possibles :**
- **Aller en haut** : si `i-1 >= 0` et `n[i-1,j] == 0` et `n[i-1,j+1] == 0`
- **Aller en bas** : si `i+1 < row` et `n[i+1,j] == 0` et `n[i+1,j+1] == 0`
- **Aller à droite** : si `j+2 < col` et `n[i,j+2] == 0`
- **Aller à gauche** : si `j-1 >= 0` et `n[i,j-1] == 0`

---

## **3. Pièces Uniques (Type 3)**

### **Identifier la pièce :**
- Une pièce unique (1x1) est détectée si :  
  - Elle occupe une seule case identifiable par sa valeur unique.

### **Actions possibles :**
- **Aller en haut** : si `i-1 >= 0` et `n[i-1,j] == 0`
- **Aller en bas** : si `i+1 < row` et `n[i+1,j] == 0`
- **Aller à droite** : si `j+1 < col` et `n[i,j+1] == 0`
- **Aller à gauche** : si `j-1 >= 0` et `n[i,j-1] == 0`

---

## **4. Pièces Carrées (Type 4)**

### **Identifier la pièce :**
- Une pièce carrée (2x2) est détectée si :  
  - `i+1 < row` et `j+1 < col` et  
    `n[i,j] == n[i+1,j] == n[i,j+1] == n[i+1,j+1]`

### **Actions possibles :**
- **Aller en haut** : si `i-1 >= 0` et `n[i-1,j] == 0` et `n[i-1,j+1] == 0`
- **Aller en bas** : si `i+2 < row` et `n[i+2,j] == 0` et `n[i+2,j+1] == 0`
- **Aller à droite** : si `j+2 < col` et `n[i,j+2] == 0` et `n[i+1,j+2] == 0`
- **Aller à gauche** : si `j-1 >= 0` et `n[i,j-1] == 0` et `n[i+1,j-1] == 0`

---

In [1]:
class Problem:
    """La classe abstraite pour un problème formel."""

    def __init__(self, initial, goal):
        self.initial = initial  # L'état initial
        self.goal = goal  # L'objectif

    def which_piece(self, state, i, j, visited):
        """
        Identifie le type de pièce à partir de la position (i, j) dans un état donné.
        Priorise les types : Carré > Verticale > Horizontale > Unique.
        Utilise `visited` pour éviter de réidentifier une partie déjà détectée.
        """
        if state[i][j] == '0' or (i, j) in visited:  # Vérifie si la case est vide ou déjà visitée
            return None
    
        piece = state[i][j]
        rows, cols = len(state), len(state[0])
    
        # Vérifie Carré
        if i+1 < rows and j+1 < cols and state[i+1][j] == piece and state[i][j+1] == piece and state[i+1][j+1] == piece:
            positions = [(i, j), (i+1, j), (i, j+1), (i+1, j+1)]
            visited.update(positions)  # Marque toutes les cases comme visitées
            return "carré", positions
    
        # Vérifie Verticale
        if i+1 < rows and state[i+1][j] == piece:
            positions = [(i, j), (i+1, j)]
            visited.update(positions)
            return "verticale", positions
    
        # Vérifie Horizontale
        if j+1 < cols and state[i][j+1] == piece:
            positions = [(i, j), (i, j+1)]
            visited.update(positions)
            return "horizontale", positions
    
        # Sinon, c'est une pièce unique
        visited.add((i, j))  # Marque cette case comme visitée
        return "unique", [(i, j)]

    def actions(self, state):
        """
        Retourne les actions possibles pour l'état donné.
        """
        actions = []
        rows, cols = len(state), len(state[0])
        visited = set()  # Pour suivre les cases déjà traitées
    
        for i in range(rows):
            for j in range(cols):
                piece_info = self.which_piece(state, i, j, visited)
                if piece_info:
                    piece_type, positions = piece_info
                    # Calcul des mouvements possibles
                    if piece_type == "verticale":
                        if i-1 >= 0 and state[i-1][j] == '0':
                            actions.append(("verticale", "UP", positions))
                        if i+2 < rows and state[i+2][j] == '0':
                            actions.append(("verticale", "DOWN", positions))
                        if j-1 >= 0 and state[i][j-1] == '0':
                            actions.append(("verticale", "LEFT", positions))
                        if j+1 < cols and state[i][j+1] == '0':
                            actions.append(("verticale", "RIGHT", positions))
    
                    elif piece_type == "horizontale":
                        if i-1 >= 0 and state[i-1][j] == '0' and state[i-1][j+1] == '0':
                            actions.append(("horizontale", "UP", positions))
                        if i+1 < rows and state[i+1][j] == '0' and state[i+1][j+1] == '0':
                            actions.append(("horizontale", "DOWN", positions))
                        if j-1 >= 0 and state[i][j-1] == '0':
                            actions.append(("horizontale", "LEFT", positions))
                        if j+2 < cols and state[i][j+2] == '0':
                            actions.append(("horizontale", "RIGHT", positions))
    
                    elif piece_type == "carré":
                        if i-1 >= 0 and state[i-1][j] == '0' and state[i-1][j+1] == '0':
                            actions.append(("carré", "UP", positions))
                        if i+2 < rows and state[i+2][j] == '0' and state[i+2][j+1] == '0':
                            actions.append(("carré", "DOWN", positions))
                        if j-1 >= 0 and state[i][j-1] == '0' and state[i+1][j-1] == '0':
                            actions.append(("carré", "LEFT", positions))
                        if j+2 < cols and state[i][j+2] == '0' and state[i+1][j+2] == '0':
                            actions.append(("carré", "RIGHT", positions))
    
                    elif piece_type == "unique":
                        if i-1 >= 0 and state[i-1][j] == '0':
                            actions.append(("unique", "UP", positions))
                        if i+1 < rows and state[i+1][j] == '0':
                            actions.append(("unique", "DOWN", positions))
                        if j-1 >= 0 and state[i][j-1] == '0':
                            actions.append(("unique", "LEFT", positions))
                        if j+1 < cols and state[i][j+1] == '0':
                            actions.append(("unique", "RIGHT", positions))
    
        return actions

    def result(self, state, action):
        """
        Applique une action à une pièce dans l'état actuel et retourne le nouvel état.
        """
        piece_type, direction, positions = action
        new_state = [row[:] for row in state]  # Crée une copie de l'état actuel
        
        for i, j in positions:
            new_state[i][j] = '0'  # Vider les anciennes positions (caractère '0')
        
        if direction == "UP":
            new_positions = [(i-1, j) for i, j in positions]
        elif direction == "DOWN":
            new_positions = [(i+1, j) for i, j in positions]
        elif direction == "LEFT":
            new_positions = [(i, j-1) for i, j in positions]
        elif direction == "RIGHT":
            new_positions = [(i, j+1) for i, j in positions]
        else:
            raise ValueError(f"Direction inconnue : {direction}")
        
        for (i, j), (new_i, new_j) in zip(positions, new_positions):
            new_state[new_i][new_j] = state[i][j]  # Remplir les nouvelles positions
        
        return new_state

    def goal_test(self, state):
        """Vérifie si l'état actuel correspond à l'objectif."""
        return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Calcule le coût du chemin."""
        return c + 1

    def value(self, state):
        """Méthode non implémentée pour des problèmes d'optimisation."""
        raise NotImplementedError



class Node:
    """
    Classe représentant un nœud dans l'arbre de recherche.
    """
    def __init__(self, state, parent=None, action=None, path_cost=0):
        """
        Initialise un nœud.
        
        :param state: L'état représenté par ce nœud.
        :param parent: Le nœud parent (par défaut None pour la racine).
        :param action: L'action qui a conduit à cet état.
        :param path_cost: Le coût du chemin pour atteindre cet état.
        """
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def expand(self, problem):
        """
        Génère les nœuds enfants pour les actions possibles dans cet état.
        
        :param problem: Une instance du problème.
        :return: Une liste de nœuds enfants.
        """
        return [
            Node(problem.result(self.state, action), self, action, self.path_cost + problem.path_cost(self.path_cost, self.state, action, problem.result(self.state, action)))
            for action in problem.actions(self.state)
        ]

    def solution(self):
        """
        Retourne la séquence d'actions pour atteindre cet état depuis la racine.
        """
        return [node.action for node in self.path()[1:]]

    def path(self):
        """
        Retourne la liste des nœuds sur le chemin depuis la racine jusqu'à ce nœud.
        """
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    def __eq__(self, other):
        """
        Vérifie si deux nœuds sont égaux (basé sur l'état).
        """
        return isinstance(other, Node) and self.state == other.state

    def __lt__(self, other):
        """
        Comparaison pour des algorithmes comme A* (basé sur le coût du chemin).
        """
        return self.path_cost < other.path_cost

    def __repr__(self):
        """
        Représentation lisible d'un nœud.
        """
        return f"<Node state={self.state} cost={self.path_cost}>"




from collections import deque

def breadth_first_search(problem):
    """
    Résout un problème donné en utilisant la recherche en largeur (BFS).
    :param problem: Une instance de la classe Problem.
    :return: Une liste d'actions pour résoudre le problème, ou None si aucune solution n'est trouvée.
    """
    # Initialisation
    node = Node(problem.initial)  # Nœud racine
    if problem.goal_test(node.state):  # Vérifie si l'état initial est déjà la solution
        return node.solution()
    
    frontier = deque([node])  # File FIFO pour BFS
    explored = set()  # Ensemble pour suivre les états visités

    while frontier:
        # Récupère le premier nœud dans la file
        node = frontier.popleft()
        explored.add(tuple(map(tuple, node.state)))  # Ajoute l'état à l'ensemble des visités

        # Génère les enfants
        for child in node.expand(problem):
            if tuple(map(tuple, child.state)) not in explored and all(child.state != n.state for n in frontier):
                if problem.goal_test(child.state):  # Vérifie si l'état enfant est l'objectif
                    return child.solution()
                frontier.append(child)  # Ajoute l'enfant à la file

    return None  # Retourne None si aucune solution n'est trouvée



NameError: name 'state' is not defined

# La class Node